# RNN计算机制

循环神经网络（recurrent neural network，简称RNN）源自于1982年由Saratha Sathasivam 提出的霍普菲尔德网络。循环神经网络，是指在全连接神经网络的基础上增加了前后时序上的关系，可以更好地处理比如机器翻译等的与时序相关的问题。

在传统的神经网络模型中，是从输入层到隐含层再到输出层，层与层之间是全连接的，每层之间的节点是无连接的。

<center><img src="https://i-blog.csdnimg.cn/blog_migrate/9d914ca33c0ab5e3899f1c43c317442e.png"></center>

一个典型的 RNN 网络架构包含一个输入，一个输出和一个神经网络单元 。和普通的前馈神经网络的区别在于：RNN 的神经网络单元不但与输入和输出存在联系，而且自身也存在一个循环 / 回路 / 环路 / 回环 (loop)。这种回路允许信息从网络中的一步传递到下一步。

>RNN的本质：上一个时刻的网络状态将会作用于（影响）到下一个时刻的网络状态

## 1.最经典的RNN公式（vanilla rnn）

隐状态ht是对“历史的压缩表示”。
给定序列输入x1, x2,...,xt，隐状态ht的更新为：
$$ h_{t}=\phi\left(W_{x h} x_{t}+W_{h h} h_{t-1}+b_{h}\right)$$

其中：
- $x_{t} \in \mathbb{R}^{d_{x}}$，$x_{t}$代表当前时刻的输入
- $h_{t-1} \in \mathbb{R}^{d_{h}}$, $h_{t-1}$代表上一时刻的记忆，它不是某个词，而是模型到目前为止对历史的总结
- $W_{x h} \in \mathbb{R}^{d_{h} \times d_{x}}$, $W_{x h}$是“输入 → 记忆”的转换规则，将当前的输入，转换成对记忆有用的形式
- $W_{h h} \in \mathbb{R}^{d_{h} \times d_{h}}$, $W_{h h}$是“记忆 → 记忆”的转换规则，它决定过去的信息要保留多少
- $b_{h} \in \mathbb{R}^{d_{h}}$, $b_{h}$是偏置
- $\phi$常见是 tanh 或 ReLU（vanilla RNN 最常用 tanh）*

>一个简单的表示：$新的记忆=tanh(当前信息+过去记忆)$

**注意**：每个时间步用的都是同一组参数$W_{x h}$, $W_{h h}$, $b_{h}$。称为参数共享，也就是说，RNN在时间维度上是同一个函数的反复应用。

这可以带来一些好处：
- 参数量与序列长度T 无关

- 能泛化到不同长度序列

- 但也导致训练时梯度要穿越很多步

## 2.时间展开

RNN原图是有环的，有环图就像你想一次性画出整个序列的计算，但它永远在转圈。

时间展开（time unrolling / unfolding）本质上是把“带环的递归计算”变成“无环的前向计算图”。

<center><img src="https://i-blog.csdnimg.cn/blog_migrate/7e3c62157f034a10c33f35cab65c5ba1.png"></center>

>为什么需要时间展开？

### (1)为了“按时间顺序”把前向算清楚（依赖链）
RNN 的前向本质就是：h1 -> h2 ->...-> ht

想算$h_t$就必须算$h_{t-1}$, 没法跳过中间步骤。

>时间展开把这种依赖结构变成明确的“链式流程”。这点对代码时间很重要，在代码里就是一个for循环

### (2)为了训练 —— BPTT（时间反向传播）
深度学习训练依靠反向传播（backprop），反向传播的前提是你有一个无环的计算图（DAG），才能按拓扑顺序做梯度回传。

原始 RNN 是有环的，没法直接在“环”上定义标准的反向传播流程。

时间展开把环拆开后，就得到得到一个标准的前馈网络（只不过层数 = 时间步数 T），于是可以做反向传播。

### ()

## 3.手写一个RNN cell

使用pytorch实现一个单步的RNN Cell，输入xt 和上一时刻$h_{t-1}$，输出ht。

In [10]:
import math
from typing import Optional, Tuple
import torch
import torch.nn as nn

class VanilaaRnnCell(nn.Module):
    """
    Vanilla (Elman) RNN Cell:
        h_t = tanh(x_t @ W_xh^T + h_{t-1} @ W_hh^T + b)

    Shapes:
        x_t:  (B, input_size)
        h_tm1:(B, hidden_size)
        h_t:  (B, hidden_size)

    Parameters are stored in (hidden_size, input_size) and (hidden_size, hidden_size),
    matching PyTorch nn.Linear weight convention for easy comparison.
    """
    def __init__(self, input_size: int, hidden_size: int, bias: bool = True):
        super().__init__()
        if input_size <= 0 or hidden_size <= 0:
            raise ValueError("input_size and hidden_size must be positive integers.")
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias

        # Weight matrices:
        # W_xh: (hidden_size, input_size)
        # W_hh: (hidden_size, hidden_size)
        self.W_xh = nn.Parameter(torch.empty(hidden_size, input_size))
        self.W_hh = nn.Parameter(torch.empty(hidden_size, hidden_size))

        if bias:
            self.b_h = nn.Parameter(torch.empty(hidden_size))
        else:
            self.register_parameter("b_h", None)

        self.reset_parameters()

    def reset_parameters(self):
        """
        A reasonable default init:
        - Use uniform in [-k, k], k = 1/sqrt(hidden_size)
        This is similar to common RNN initializations.
        """
        k = 1.0 / math.sqrt(self.hidden_size)
        nn.init.uniform_(self.W_xh, -k, k)
        nn.init.uniform_(self.W_hh, -k, k)
        if self.bias is not None:
            nn.init.uniform_(self.b_h, -k, k)

    def forward(self, x_t: torch.Tensor, h_tm1: torch.Tensor) -> torch.Tensor:
        """
        Single-step forward.

        Args:
            x_t:   (B, input_size)
            h_tm1: (B, hidden_size)

        Returns:
            h_t:   (B, hidden_size)
        """
        if x_t.dim() != 2:
            raise ValueError(f"x_t must be 2D (B, input_size), got shape {tuple(x_t.shape)}")
        if h_tm1.dim() != 2:
            raise ValueError(f"h_tm1 must be 2D (B, hidden_size), got shape {tuple(h_tm1.shape)}")
        if x_t.size(1) != self.input_size:
            raise ValueError(f"x_t second dim must be input_size={self.input_size}, got {x_t.size(1)}")
        if h_tm1.size(1) != self.hidden_size:
            raise ValueError(f"h_tm1 second dim must be hidden_size={self.hidden_size}, got {h_tm1.size(1)}")
        if x_t.size(0) != h_tm1.size(0):
            raise ValueError(f"Batch sizes must match, got {x_t.size(0)} vs {h_tm1.size(0)}")
        
        # (B, hidden) = (B, input) @ (hidden, input)^T
        x_part = x_t @ self.W_xh.t()
        # (B, hidden) = (B, hidden) @ (hidden, hidden)^T
        h_part = h_tm1 @ self.W_hh.t()

        preact = x_part + h_part
        if self.b_h is not None:
            preact = preact + self.b_h # broadcasts (hidden,) over batch
        
        h_t = torch.tanh(preact)
        return h_t
    
    #时间展开
    @torch.no_grad
    def forward_sequence(
        self,
        x: torch.Tensor,
        h0: Optional[torch.Tensor] = None,
        batch_first: bool = True
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Convenience function to run through a full sequence for testing.

        Args:
            x: (B, T, input_size) if batch_first else (T, B, input_size)
            h0: (B, hidden_size) optional, defaults to zeros
            batch_first: whether x is batch-first

        Returns:
            h_all: (B, T, hidden_size) if batch_first else (T, B, hidden_size)
            h_T:   (B, hidden_size)
        """
        if x.dim() != 3:
            raise ValueError(f"x must be 3D, got shape {tuple(x.shape())}")        
        
        if batch_first:
            B, T, D = x.shape
            x_seq = x
        else:
            T, B, D = x.shape
            x_seq = x.transpose(0, 1) #-> (B, T, D)
        
        if D != self.input_size:
            raise ValueError(f"input last dim must be input_size={self.input_size}, got {D}")
        
        if h0 is None:
            h_t = torch.zeros(B, self.hidden_size, device=x.device, dtype=x.dtype)
        else:
            if h0.shape != (B, self.hidden_size):
                raise ValueError(f"h0 must be shape (B, hidden_size) = ({B}, {self.hidden_size})")
            else:
                h_t = h0

        h_list = []
        for t in range(T):
            h_t = self.forward(x_seq[:, t, :], h_t)
            h_list.append(h_t)
        
        h_all = torch.stack(h_list, dim=1) #(B, T, hidden)
        if not batch_first:
            h_all = h_all.transpose(0, 1) #back to (T, B, hidden)
        
        return h_all, h_t
    
#验证cell是否与手写的时间步循环计算结果是否一致
def _quick_test():
    torch.manual_seed(42)

    B, T, Din, H = 2, 4, 3, 5
    x = torch.randn(B, T, Din)

    cell = VanilaaRnnCell(input_size=Din, hidden_size=H, bias=True)

    h_all, h_t = cell.forward_sequence(x, batch_first=True)
    print("h_all shape: ", h_all.shape) #(B, T, H)
    print("h_t shape: ", h_t.shape) #(B, H)

    #compare one step with manual call
    h0 = torch.zeros(B, H)
    h1 = cell(x[:, 0, :], h0)

    # 断言（assert）：
    # 检查“手动算出来的 h1”
    # 是否和“循环版本里存下来的第 0 个时间步输出 h_all[:, 0, :]”几乎相等
    # atol=1e-6 表示允许的数值误差（浮点误差）
    # 如果不相等，就报错并显示 "Step output mismatch!"
    assert torch.allclose(h1, h_all[:, 0, :], atol=1e-6), "Step output mismatch"
    
    print("Quick test passed.")
    # 如果没有触发 assert，说明结果一致
    # 打印提示：快速测试通过

if __name__ == "__main__":
    _quick_test()

h_all shape:  torch.Size([2, 4, 5])
h_t shape:  torch.Size([2, 5])
Quick test passed.


## 4.防止参数爆炸

我们注意到代码中有一个函数reset_parameters(self), 把 W_xh、W_hh、b 初始化到 [-k, k], 这是为什么呢？

先看公式：
$$ h_{t}=\phi\left(W_{x h} x_{t}+W_{h h} h_{t-1}+b_{h}\right)$$

如果W初始化很大，h_t 一开始就会变得很大 → 过非线性（tanh/sigmoid）会饱和 → 梯度变很小 → 学不动；或者反复递推时数值越滚越大 → 爆炸

如果W初始化很小，h_t 会非常小，反复乘 W_hh 会越乘越小 → 消失

>所以我们想要W 的尺度刚刚好，让 h 的尺度在时间上传播时比较稳定。

**下面我们来推导为什么k = 1/sqrt(hidden_size)**

设 h_{t-1} 的每个分量都像“平均大小差不多”的随机数。
W_hh h_{t-1} 的某个输出分量是：
$$\sum_{i=1}^{H} W_{j i} h_{i}$$

这是一堆随机项相加。相加项越多（H 越大），总的波动就越大，除非你把每个 W_{ji} 的尺度调小。

更具体一点（非常常见的近似）：

假设 W_{ji} 独立、均值 0，方差是 Var(W)

假设 h_i 独立、均值 0，方差是 Var(h)

那么和的方差大约是：
$$\operatorname{Var}\left(\sum_{i=1}^{H} W_{j i} h_{i}\right) \approx \sum_{i=1}^{H} \operatorname{Var}\left(W_{j i} h_{i}\right)=H \cdot \operatorname{Var}(W) \cdot \operatorname{Var}(h)$$

我们希望输出方差和输入方差差不多（别越来越大或越来越小），即：
$$H \cdot \operatorname{Var}(W) \approx 1 \Rightarrow \operatorname{Var}(W) \approx \frac{1}{H}$$

这就是关键：
>权重的方差应当随 hidden_size H 增大而按 1/H 缩小。

可以使用均匀分布初始化，也可以使用正态分布初始化：

均匀分布 U(-k, k) 的方差是：$\operatorname{Var}=\frac{k^{2}}{3}$

正态分布 N(0, σ²) 的方差是: $\operatorname{Var}=\sigma^{2}$

我们的代码里用的是均匀分布，如果我们想要 Var(W) ≈ 1/H，那：

$$\frac{k^{2}}{3} \approx \frac{1}{H} \Rightarrow k \approx \sqrt{\frac{3}{H}}$$

常数因子差一点影响不大，训练会自己调整,选 1/sqrt(H) 更保守一点（范围稍小一点），在 RNN 里常常更稳。

那么正态分布的$\sigma$值该取多少就留给各位自己去算了。

这里有个细节，我们把bias也放在了[-k,k],是为了：

避免一开始就强偏置到某个方向（比如 tanh 一直在正区间）

让 pre-activation（线性部分）整体尺度一致，更稳定

所以bias 通常也用相似尺度的初始化，保持“整体输入到非线性之前”的数值范围合理。

## 5.正交初始化

很多现代 RNN 还会对 W_hh 用 正交初始化，进一步让谱半径接近 1，更利于长序列梯度传播。

对于RNN的公式有：
$$h_{t} \approx W_{h h} h_{t-1} \Rightarrow h_{T} \approx W_{h h}^{T} h_{0}$$

**重点：**同一个W_hh被乘了 T 次（这里的上标 T 是次数，不是转置）。

如果W_hh在某个方向上“放大一点点”> 1;乘很多次后会 指数级放大 → 前向值爆炸，反向梯度也爆炸

如果W_hh在某个方向上“缩小一点点”< 1;乘很多次后会 指数级缩小 → 变成 0，梯度消失

>RNN 在时间上传播信息 = 多次连乘同一个矩阵；连乘最怕“略大于 1”或“略小于 1”。

如果 Q 是正交矩阵：$Q^{\top} Q=I$

那么对任意向量 v：$\|Q v\|=\|v\|$

**正交变换 = 纯旋转/镜像，不拉伸不压缩。**

如果把W_hh初始化成正交矩阵Q：
$$h_{T} \approx Q^{T} h_{0}$$

- 不改变向量长度

- 谱半径 = 1

- 时间连乘不容易爆炸 / 消失

因为每次乘都不改变长度，所以“能量”不会在时间上莫名其妙爆炸或消失。

正交矩阵的特征值都落在单位圆上（复平面上模长为 1 的圆），因此：
$$\rho(Q)=1$$

也就是说，正交初始化天然就把谱半径钉在 1 附近（理想情况下就是 1）。

谱半径的定义是：$\rho(W)=\max _{i}\left|\lambda_{i}\right|$，其中$\lambda_{i}$是矩阵W的特征值。

>正交初始化比均匀分布、正态分布等随机初始化更适合RNN，